In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import mwparserfromhell
from dgnutils import *
# sys.path.append("/Users/nish/development/git/etymology_explorer/modules")
sys.path.append("/home/ubuntu/projects/etymology_explorer/modules")
from ety_utils import * # imports wikiextractor
u = os.environ['RDS_ETY_USER']; p = os.environ['RDS_ETY_PASSWORD']; h = os.environ['RDS_ETY_HOST']
session=requests.session()
test_conn, test_cursor = connect('etymology_explorer_test', user=u, password=p, host=h)
# dev_conn, dev_cursor = connect('etymology_explorer_dev', user=u, password=p, host=h)
stage_conn, stage_cursor = connect('etymology_explorer_staging', user=u, password=p, host=h)

10/02/20 dgnutils update loaded! Added getFloat


# Full load test

In [ ]:
# Create test xml
# head -176497 enwiktionary-latest-pages-articles.xml > test.xml
# echo "</mediawiki>" >> test.xml
# wc -l test.xml
# tail -100 test.xml
# cd ..
# ./WikiExtractor.py --processes 4 -o output_test --json --no_templates --lists --sections input/test.xml

In [2]:
wp = WikiProcessor(
    '/home/ubuntu/projects/wikiextractor/', 
    channel='test', # Test makes self.test = true
#     channel='dev', 
    store_intermediates=True,
)

In [3]:
wp.load_wl_2_id_values()

In [6]:
processed_wikidump = wp.load_wikidump_etymologies(); len(processed_wikidump)

977

In [7]:
en_etys_dl, wp.en_dict = wp.create_and_insert_mysql_entries(processed_wikidump) # no cache

Step 0/1: 4 results


In [ ]:
test_cursor.e('SELECT COUNT(*) FROM etymologies')

# Add links to etymologies 

In [ ]:
# Do a hack
# Go through all new etymologies, and find instances where the expanded text matches to one of the existing etymologies, then add that to the etymology
# Search through each unmatched etymology, look for (language, word) that links to an _id.

In [4]:
# Manually get all the etymologies
# DO NOT RESET DATABASE
wp = WikiProcessor('/home/ubuntu/projects/wikiextractor/', channel='staging', )
wp.load_wl_2_id_values()
wp.load_language_dict()
log_level('d')

## Get en_etys_dl

In [5]:
wp.load_wl_2_id_values() # 17s
wp.load_language_dict() #1s

INFO:root:Creating wl_2_id dictionary...


In [6]:
processed_wikidump = wp.load_wikidump_etymologies(); len(processed_wikidump) # 1.5m on r4.2xl

INFO:root:Processing wikiextractor directory /home/ubuntu/projects/wikiextractor/output/
DEBUG:root:Processing wikiextractor directory: AA
DEBUG:root:Processing wikiextractor directory: AB
DEBUG:root:Processing wikiextractor directory: AC
DEBUG:root:Processing wikiextractor directory: AD
DEBUG:root:Processing wikiextractor directory: AE
DEBUG:root:Processing wikiextractor directory: AF
DEBUG:root:Processing wikiextractor directory: AG
DEBUG:root:Processing wikiextractor directory: AH
DEBUG:root:Processing wikiextractor directory: AI
DEBUG:root:Processing wikiextractor directory: AJ
DEBUG:root:Processing wikiextractor directory: AK
DEBUG:root:Processing wikiextractor directory: AL
DEBUG:root:Processing wikiextractor directory: AM
DEBUG:root:Processing wikiextractor directory: AN
DEBUG:root:Processing wikiextractor directory: AO
INFO:root:Finished on AO, 47. Found 6380374 words


In [12]:
en_conns_dl, en_etys_dl, en_pos_dl, en_prons_dl, en_defs_dl, etys_dl = wp.create_mysql_data_from_processed_wikiextraction_data(processed_wikidump, log=True)

INFO:root:Found 7072848 new connections to insert
INFO:root:Found 1453725 new etymologies to insert
INFO:root:Found 6699058 new pos to insert
INFO:root:Found 1424616 new pronunciations to insert
INFO:root:Found 9060557 new definitions to insert


In [27]:
parsed_etymologies_except_conns = multi_parse_wikitext_sentences(
    [e['wikitext'] for e in en_etys_dl], 
    cache_file=wp.cache_path+'ety.wik' if wp.cache_path else None,
    ignore_connection_forming=True,
)
en_etys_dl = [{**z[0], 'wikitext':z[1]} for z in zip(en_etys_dl, parsed_etymologies_except_conns)]

DEBUG:root:Generating list of used wikitexts...
DEBUG:root:Using cache file: /home/ubuntu/projects/wikiextractor/input/ety.wik...
DEBUG:root:Found 1957671 wikitexts in /home/ubuntu/projects/wikiextractor/input/ety.wik...
DEBUG:root:All wikitext accounted for...
INFO:root:Replacing wikitext in the sentences...


In [28]:
parsed_etymologies = multi_parse_wikitext_sentences(
    [e['wikitext'] for e in en_etys_dl], 
    cache_file=wp.cache_path+'ety.wik' if wp.cache_path else None,
    ignore_connection_forming=False,
)
en_etys_dl = [{**z[0], 'etymology':z[1]} for z in zip(en_etys_dl, parsed_etymologies)]

DEBUG:root:Generating list of used wikitexts...
DEBUG:root:Using cache file: /home/ubuntu/projects/wikiextractor/input/ety.wik...
DEBUG:root:Found 1957671 wikitexts in /home/ubuntu/projects/wikiextractor/input/ety.wik...
DEBUG:root:There are 477 to gather...
DEBUG:root:There are 477 to gather...
INFO:root:Generating groups of wikitexts for api calls over 5 steps...
INFO:root:Performing API calls on wikitext_groupings over 1 steps...


Step 0/1: 

DEBUG:root:Saving cache file


477 results


INFO:root:Replacing wikitext in the sentences...


## start from en_etys_dl

In [5]:
# Checkpoint
# pickle.dump(en_etys_dl, open('en_etys_dl', 'wb+'))
en_etys_dl = pickle.load(open('en_etys_dl', 'rb'))

In [6]:
len(en_etys_dl), en_etys_dl[0]

(1453725,
 {'_id': 10149,
  'word': 'dictionary',
  'language_name': 'English',
  'entry_id': 0,
  'wikitext': 'Borrowed from {{bor|en|ML.|dictionarium}}, from {{der|en|la|dictionarius}}, from {{m|la|dictio||speaking}}, from {{m|la|dictus}}, perfect past participle of {{m|la|dīcō||speak}} + {{m|la|-arium||room, place}}.',
  'etymology': 'Borrowed from Medieval Latin dictionarium, from Latin dictionarius, from dictio (“speaking”), from dictus, perfect past participle of dīcō (“speak”) + -arium (“room, place”).'})

In [29]:
wp.wl_2_id[('dictionarium', 'Latin')]

47136

In [25]:
wp.getNodesFromTemplate('{{bor|en|ML.|dictionarium}}', 'root', allow_non_connections=True)
# getTemplateInfo('{{bor|en|ML.|dictionarium}}')

[{'word': 'dictionarium', 'language': 'Medieval Latin'}]

In [13]:
templates = mwparserfromhell.parse(en_etys_dl[0]['wikitext']).filter_templates(); templates
[wp.getNodesFromTemplate(t, 'root') for t in templates]

['{{bor|en|ML.|dictionarium}}',
 '{{der|en|la|dictionarius}}',
 '{{m|la|dictio||speaking}}',
 '{{m|la|dictus}}',
 '{{m|la|dīcō||speak}}',
 '{{m|la|-arium||room, place}}']

In [ ]:
for en_ety in en_etys_dl:
    
wp.load

In [ ]:
# Check for multiple entries

In [32]:
[a['etymology'] for a in all_entry_etymologies if 'Borrowed from Medieval Latin dictionarium' in a['etymology']]

['English dictionary : Borrowed from Medieval Latin dictionarium, from Latin dictionarius, from dictio (“speaking”), from dictus, perfect past participle of dīcō (“speak”) + -arium (“room, place”).',
 'Portuguese dicionário : Borrowed from Medieval Latin dictionarium, from Latin dictionarius, from dictiō (“speaking”) + -arium (“room, place”).']

In [ ]:
processed_wikidump

In [23]:
# 12.8s
all_entry_etymologies = stage_cursor.d('SELECT entry_id, etymology FROM entry_etymologies')

# Get entries info from wikitext

In [ ]:
# Get all the wikitext entries (for now just a sample)

# Find ones that have a gloss, or definition
# Find the word and language for that wikitext, then the id
# See if that id has an existing definition or entry